# AIAP Project Score MLP for Deployment

## We pick the best model to be RF Regressor and we will define our pipeline for deployment using make_pipeline in six stages:

(1) Get user input for url of train data, use SQL Lite to retrieve data and pass to dataframe 'df'

(2) Create a ColumnTransformer that will transform the data into numerical variables and categorical variables, including ONE-Hot Encoder where required.
'number_of_siblings' - num 
'direct_admission' - cat (boolean: Yes, No)
'learning_style' - cat (boolean: Visual, Auditory)
'tuition' - cat (boolean: Yes, No)
'n_male' - num
'n_female' - num
'hours_per_week' - num
'attendance_rate' - num
'CCA_O' - cat (ONE-Hot encoder, drop CCA_A, CCA_S, CCA_C)

(3)Perform data preprocessing: 
- Drop the columns that are not required for model training, leaving 9 features and y variable
- Remove NaN values or Impute values where appropriate

(4) Scale the data using StandardScaler()

(5) Pass through the RF Regressor to train the model 

(6) Get user input for url of data for prediction. Run the model and get the predicted value for final_test scores. Export final_test scores to csv file on local file_path

In [1]:

!pip install pipreqs



In [2]:
# pip freeze > requirements.txt  ## or go to Terminal and find dir using cd, type pipreqs.  
## Requirements.txt will autogenerate and save in the same folder


In [28]:
# basic

import numpy as np
import pandas as pd
import datetime as dt

import math
from enum import Enum
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

# mute warnings
import warnings
warnings.filterwarnings("ignore")

# scikit-learn
from sklearn import metrics
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# import category encoders
!pip install category_encoders
import category_encoders as ce

# ML Algo
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  RandomForestRegressor

# SQL Lite
import sqlite3
import requests
import sqlalchemy



In [29]:
# pip freeze > requirements.txt

In [30]:
from configparser import ConfigParser
config = ConfigParser()
config.read("config.ini")

import configparser

def read_config_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    return config

if __name__ == "__main__":
    config_file_path = "config.ini"
    config = read_config_file(config_file_path)

    # Access configuration settings
    train_url = config.get("DEFAULT", "train_url")
    predict_url = config.get("DEFAULT", "predict_url")
    file_path = config.get("DEFAULT", "file_path")

    print("train_url", train_url)
    print("predict_url", predict_url)
    print("file_path", file_path)




train_url https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db
predict_url https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db
file_path C:\...prediction.csv


In [31]:
# Ask User for URL to data file and fetch data from URL
# https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db

def get_url_from_user():
    """
    Function to ask the user to input a URL.
    
    Returns:
    url (str): The URL inputted by the user.
    """
    url = train_url # input("Please enter the URL to your Training Data: ")
    return url

# Example usage:
if __name__ == "__main__":
    user_url = get_url_from_user()
    print("You entered:", user_url)
    
    response = requests.get(user_url)
    score_data = sqlite3.connect('score.db')
    ## table name is known
    db_name = "score.db"
    table_name = "score"  ## insert table name
    engine = sqlalchemy.create_engine('sqlite:///%s'% db_name, execution_options={"sqlite_raw_colnames":True})
    print("Table Name:", table_name)
    df = pd.DataFrame()
    df = pd.read_sql_table(table_name, engine)
    df = df.dropna()
    df = df.drop_duplicates(subset='student_id')
    df.columns = [str(col) for col in df.columns]

    print(df)



You entered: https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db
Table Name: score
       index  number_of_siblings direct_admission     CCA learning_style  \
0          0                   0              Yes  Sports         Visual   
1          1                   2               No  Sports       Auditory   
2          2                   0              Yes    None         Visual   
4          4                   0               No  Sports       Auditory   
5          5                   0               No    Arts         Visual   
...      ...                 ...              ...     ...            ...   
15895  15895                   1               No   Clubs         Visual   
15896  15896                   1              Yes    None       Auditory   
15897  15897                   1              Yes  Sports       Auditory   
15898  15898                   1               No   Clubs         Visual   
15899  15899                   2              Yes    

In [32]:
# Load df

df

,index,number_of_siblings,direct_admission,CCA,learning_style,student_id,gender,tuition,final_test,n_male,n_female,age,hours_per_week,attendance_rate,sleep_time,wake_time,mode_of_transport,bag_color
0,0,0,Yes,Sports,Visual,ACN2BE,Female,No,69.0,14.0,2.0,16.0,10.0,91.0,22:00,6:00,private transport,yellow
1,1,2,No,Sports,Auditory,FGXIIZ,Female,No,47.0,4.0,19.0,16.0,7.0,94.0,22:30,6:30,private transport,green
2,2,0,Yes,None,Visual,B9AI9F,Male,No,85.0,14.0,2.0,15.0,8.0,92.0,22:30,6:30,private transport,white
4,4,0,No,Sports,Auditory,AXZN2E,Male,No,66.0,24.0,3.0,16.0,7.0,95.0,21:30,5:30,public transport,yellow
5,5,0,No,Arts,Visual,BA6R14,Female,No,57.0,9.0,12.0,15.0,11.0,96.0,22:30,6:30,private transport,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15895,15895,1,No,Clubs,Visual,XPECN2,Female,No,56.0,12.0,14.0,16.0,9.0,96.0,22:00,6:00,private transport,black
15896,15896,1,Yes,None,Auditory,7AMC7S,Male,Yes,85.0,17.0,5.0,16.0,7.0,91.0,22:30,6:30,private transport,white
15897,15897,1,Yes,Sports,Auditory,XKZ6VN,Female,Yes,76.0,7.0,10.0,15.0,7.0,93.0,23:00,7:00,walk,red
15898,15898,1,No,Clubs,Visual,2OU4UQ,Male,Yes,45.0,18.0,12.0,16.0,3.0,94.0,23:00,7:00,walk,yellow


In [33]:
# There are now 19 columns , including dummies. Move column 'final_test' to the end
df = df[[col for col in df if col != 'final_test'] + ['final_test']]
df

,index,number_of_siblings,direct_admission,CCA,learning_style,student_id,gender,tuition,n_male,n_female,age,hours_per_week,attendance_rate,sleep_time,wake_time,mode_of_transport,bag_color,final_test
0,0,0,Yes,Sports,Visual,ACN2BE,Female,No,14.0,2.0,16.0,10.0,91.0,22:00,6:00,private transport,yellow,69.0
1,1,2,No,Sports,Auditory,FGXIIZ,Female,No,4.0,19.0,16.0,7.0,94.0,22:30,6:30,private transport,green,47.0
2,2,0,Yes,None,Visual,B9AI9F,Male,No,14.0,2.0,15.0,8.0,92.0,22:30,6:30,private transport,white,85.0
4,4,0,No,Sports,Auditory,AXZN2E,Male,No,24.0,3.0,16.0,7.0,95.0,21:30,5:30,public transport,yellow,66.0
5,5,0,No,Arts,Visual,BA6R14,Female,No,9.0,12.0,15.0,11.0,96.0,22:30,6:30,private transport,red,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15895,15895,1,No,Clubs,Visual,XPECN2,Female,No,12.0,14.0,16.0,9.0,96.0,22:00,6:00,private transport,black,56.0
15896,15896,1,Yes,None,Auditory,7AMC7S,Male,Yes,17.0,5.0,16.0,7.0,91.0,22:30,6:30,private transport,white,85.0
15897,15897,1,Yes,Sports,Auditory,XKZ6VN,Female,Yes,7.0,10.0,15.0,7.0,93.0,23:00,7:00,walk,red,76.0
15898,15898,1,No,Clubs,Visual,2OU4UQ,Male,Yes,18.0,12.0,16.0,3.0,94.0,23:00,7:00,walk,yellow,45.0


In [34]:
## Apply Train-test split to data
from sklearn.model_selection import train_test_split
X = pd.DataFrame()
X = df.iloc[ : , :-1]
y = pd.DataFrame()
y = df.iloc[ : , -1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (11108, 17) (11108, 1)
Test set: (2777, 17) (2777, 1)


In [35]:
X_train

,index,number_of_siblings,direct_admission,CCA,learning_style,student_id,gender,tuition,n_male,n_female,age,hours_per_week,attendance_rate,sleep_time,wake_time,mode_of_transport,bag_color
12953,12953,1,No,SPORTS,Auditory,ND2A2E,Female,Yes,17.0,11.0,15.0,19.0,92.0,23:00,7:00,walk,red
7209,7209,2,No,Sports,Auditory,2C4PV1,Male,Yes,14.0,4.0,16.0,11.0,91.0,22:30,6:30,private transport,blue
2446,2446,0,Yes,Clubs,Visual,V6EAW3,Female,Yes,4.0,12.0,15.0,8.0,97.0,22:00,6:00,private transport,black
7189,7189,0,No,Sports,Visual,UHFPEM,Male,Yes,15.0,10.0,15.0,10.0,99.0,22:00,6:00,private transport,blue
2996,2996,1,No,Arts,Visual,O2VTM5,Male,No,21.0,9.0,15.0,6.0,95.0,21:30,5:30,public transport,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6666,6666,0,No,Arts,Visual,AR6YN0,Male,Yes,27.0,1.0,6.0,11.0,93.0,21:00,5:00,public transport,yellow
751,751,1,Yes,Clubs,Auditory,ZF4NK4,Female,Yes,1.0,16.0,15.0,4.0,91.0,22:00,6:00,private transport,green
12085,12085,0,No,Arts,Auditory,UCPBMW,Female,Yes,3.0,19.0,5.0,8.0,91.0,21:00,5:00,public transport,red
9377,9377,0,No,Clubs,Auditory,RS04X5,Male,Yes,10.0,11.0,15.0,14.0,100.0,23:00,7:00,walk,green


In [36]:
y_train

,final_test
12953,58.0
7209,55.0
2446,81.0
7189,96.0
2996,47.0
...,...
6666,73.0
751,76.0
12085,90.0
9377,75.0


In [37]:
## import BaseEstimator. Grid Search and Pipelines: BaseEstimator is crucial for functionalities like grid search (GridSearchCV) and pipelines (Pipeline). Grid search uses get_params() to explore a range of hyperparameters, while pipelines rely on fit() and predict() to chain together multiple estimators.

from sklearn.base import BaseEstimator

## create class CCAEncoder to apply ONE_Hot Encoder to CCA = None

class CCAEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, df):
        df['CCA_O'] = (df['CCA'] == 'None')*1
        df['learning_style_visual'] = (df['learning_style'] == 'Visual')*1
        df['tuition_Yes'] = (df['tuition'] == 'Yes')*1
        df['direct'] = (df['direct_admission'] == 'Yes')*1

        return df


In [38]:
## Where necessary, impute NaN values with strategy='mean' or 'most frequent' or others
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')



In [39]:
# custom transformer must have methods fit and transform
# Scale the data in the column Item_MRP
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['sleep_time',
                                                                        'wake_time',
                                                                        'gender',
                                                                        'student_id',
                                                                        'age',
                                                                        'tuition',
                                                                        'direct_admission',
                                                                        'learning_style',
                                                                        'mode_of_transport',
                                                                        'CCA',
                                                                        'bag_color'])])
                                           


In [40]:
model_pipe = make_pipeline(
    CCAEncoder(), # One_Hot Encoder for CCA = None
    pre_process,
    StandardScaler(),
    RandomForestRegressor(max_depth=10)
)


In [41]:
model_pipe

Pipeline(steps=[('ccaencoder', CCAEncoder()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['sleep_time', 'wake_time',
                                                   'gender', 'student_id',
                                                   'age', 'tuition',
                                                   'direct_admission',
                                                   'learning_style',
                                                   'mode_of_transport', 'CCA',
                                                   'bag_color'])])),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor(max_depth=10))])

In [42]:
# Train the pipeline
model_pipe.fit(X_train, y_train)

Pipeline(steps=[('ccaencoder', CCAEncoder()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['sleep_time', 'wake_time',
                                                   'gender', 'student_id',
                                                   'age', 'tuition',
                                                   'direct_admission',
                                                   'learning_style',
                                                   'mode_of_transport', 'CCA',
                                                   'bag_color'])])),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor(max_depth=10))])

In [43]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# Test the pipeline
y_pred = model_pipe.predict(X_test)

# Evaluate performance
r2_score_test = r2_score(y_test, y_pred)
rmse_test = (np.sqrt(mean_squared_error(y_test, y_pred)))

print("Model_pipe r2_score_test: ", r2_score_test)
print("Model_pipe rmse_test: ", rmse_test)

Model_pipe r2_score_test:  0.7020326527473899
Model_pipe rmse_test:  7.663380824032008


In [44]:

# Ask User for URL to prediction data file and fetch prediction data from URL
# https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db

def get_url_from_user():
    """
    Function to ask the user to input a URL.
    
    Returns:
    url (str): The URL inputted by the user.
    """
    url = predict_url # input("Please enter the URL to your Prediction Data: ")
    return url

# Example usage:
if __name__ == "__main__":
    user_url = get_url_from_user()
    print("You entered:", user_url)

    response = requests.get(user_url)
    score_data = sqlite3.connect('score.db')
    ## table name is known
    db_name = "score.db"
    table_name = "score"  ## insert table name
    engine = sqlalchemy.create_engine('sqlite:///%s'% db_name, execution_options={"sqlite_raw_colnames":True})
    print("Table Name:", table_name)
    df1 = pd.DataFrame()
    df1 = pd.read_sql_table(table_name, engine)
    df1 = df.dropna()
    df1 = df.drop_duplicates(subset='student_id')
    df1.columns = [str(col) for col in df.columns]

    print(df1)


You entered: https://techassessment.blob.core.windows.net/aiap-preparatory-bootcamp/score.db
Table Name: score
       index  number_of_siblings direct_admission     CCA learning_style  \
0          0                   0              Yes  Sports         Visual   
1          1                   2               No  Sports       Auditory   
2          2                   0              Yes    None         Visual   
4          4                   0               No  Sports       Auditory   
5          5                   0               No    Arts         Visual   
...      ...                 ...              ...     ...            ...   
15895  15895                   1               No   Clubs         Visual   
15896  15896                   1              Yes    None       Auditory   
15897  15897                   1              Yes  Sports       Auditory   
15898  15898                   1               No   Clubs         Visual   
15899  15899                   2              Yes    

In [45]:
# There are now 19 columns , including dummies. 'final_test' column should be blank, so move column 'final_test' to the end
df1 = df1[[col for col in df if col != 'final_test'] + ['final_test']]

## Drop final_test from Prediction Data 
X = pd.DataFrame()
X = df1.iloc[ : , :-1]
X.set_index('student_id', inplace=True)  ## we want to preserve student_id so we can match our predicted final_test score back to a specific student
X


,index,number_of_siblings,direct_admission,CCA,learning_style,gender,tuition,n_male,n_female,age,hours_per_week,attendance_rate,sleep_time,wake_time,mode_of_transport,bag_color
student_id,,,,,,,,,,,,,,,,
ACN2BE,0,0,Yes,Sports,Visual,Female,No,14.0,2.0,16.0,10.0,91.0,22:00,6:00,private transport,yellow
FGXIIZ,1,2,No,Sports,Auditory,Female,No,4.0,19.0,16.0,7.0,94.0,22:30,6:30,private transport,green
B9AI9F,2,0,Yes,None,Visual,Male,No,14.0,2.0,15.0,8.0,92.0,22:30,6:30,private transport,white
AXZN2E,4,0,No,Sports,Auditory,Male,No,24.0,3.0,16.0,7.0,95.0,21:30,5:30,public transport,yellow
BA6R14,5,0,No,Arts,Visual,Female,No,9.0,12.0,15.0,11.0,96.0,22:30,6:30,private transport,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XPECN2,15895,1,No,Clubs,Visual,Female,No,12.0,14.0,16.0,9.0,96.0,22:00,6:00,private transport,black
7AMC7S,15896,1,Yes,None,Auditory,Male,Yes,17.0,5.0,16.0,7.0,91.0,22:30,6:30,private transport,white
XKZ6VN,15897,1,Yes,Sports,Auditory,Female,Yes,7.0,10.0,15.0,7.0,93.0,23:00,7:00,walk,red


In [46]:
predict_pipe = make_pipeline(
    CCAEncoder(), # One_Hot Encoder for CCA = None
    pre_process,
    StandardScaler(),
    model_pipe.predict(X),
    print(model_pipe.predict(X))
    
)


[73.53738406 54.1329678  81.93924881 ... 75.22022476 46.7397553
 80.12483142]


In [47]:
y_prediction = model_pipe.predict(X)
y_p = pd.DataFrame()
y_p = pd.DataFrame(y_prediction)
X_index = pd.DataFrame(X.index)  ## put the student_id back to the predicted final_test scores
y_df = pd.concat([X_index, y_p], axis=1)



In [48]:
y_df

,student_id,0
0,ACN2BE,73.537384
1,FGXIIZ,54.132968
2,B9AI9F,81.939249
3,AXZN2E,70.638862
4,BA6R14,59.651274
...,...,...
13880,XPECN2,55.392689
13881,7AMC7S,78.922518
13882,XKZ6VN,75.220225
13883,2OU4UQ,46.739755


In [49]:
# Ask User for URL for file_path for saving y_prediction df
# C:\Users\hclee\Desktop\prediction.csv

def get_url_from_user():
    """
    Function to ask the user to input a file_path.
    
    Returns:
    url (str): The URL inputted by the user.
    """
    url = file_path # input("Please enter the file_path with final name ending in csv on your local drive where you want to save your final_test score prediction")
    
    return url

# Example usage:
if __name__ == "__main__":
    file_path = get_url_from_user()
    print("You entered:", file_path)


You entered: C:\...prediction.csv


In [50]:

y_df.to_csv(file_path, index=False)

# retrieve y_df to make sure it was saved
y_df_final = pd.read_csv(file_path)
y_df_final

,student_id,0
0,ACN2BE,73.537384
1,FGXIIZ,54.132968
2,B9AI9F,81.939249
3,AXZN2E,70.638862
4,BA6R14,59.651274
...,...,...
13880,XPECN2,55.392689
13881,7AMC7S,78.922518
13882,XKZ6VN,75.220225
13883,2OU4UQ,46.739755


In [51]:
print(file_path)

C:\...prediction.csv


In [25]:
import joblib
joblib.dump(model_pipe, "model_pipe.joblib")

['model_pipe.joblib']

In [26]:
model_pipe2 = joblib.load("model_pipe.joblib")
model_pipe2

Pipeline(steps=[('ccaencoder', CCAEncoder()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['sleep_time', 'wake_time',
                                                   'gender', 'student_id',
                                                   'age', 'tuition',
                                                   'direct_admission',
                                                   'learning_style',
                                                   'mode_of_transport', 'CCA',
                                                   'bag_color'])])),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor(max_depth=10))])

In [27]:
# %whos

Variable                        Type                 Data/Info
--------------------------------------------------------------
BaseEstimator                   type                 <class 'sklearn.base.BaseEstimator'>
CCAEncoder                      type                 <class '__main__.CCAEncoder'>
ColumnTransformer               ABCMeta              <class 'sklearn.compose._<...>ormer.ColumnTransformer'>
ConfigParser                    ABCMeta              <class 'configparser.ConfigParser'>
DecisionTreeClassifier          ABCMeta              <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
Enum                            EnumMeta             <enum 'Enum'>
LinearRegression                ABCMeta              <class 'sklearn.linear_mo<...>._base.LinearRegression'>
LogisticRegression              type                 <class 'sklearn.linear_mo<...>stic.LogisticRegression'>
Pipeline                        ABCMeta              <class 'sklearn.pipeline.Pipeline'>
RandomForestRegresso